In [ ]:
import os
from dotenv import load_dotenv
import psycopg2
import requests
from typing import Dict, List
import logging 
# Load the environment variables from .env file
load_dotenv('/home/wjones/CC/Capstone/tbd2/LACCTiC/.env', override=True)
logging.basicConfig(filename='db_insert.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Get the database credentials from environment variables
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASS')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

# Connect to the database
db_params = {
    'dbname': db_name,
    'user': db_user,
    'password' : db_password,
    'host' : db_host,
    'port' : db_port
}

In [ ]:
def connect_db(db_params):
    """Connect to the PostgreSQL database server."""
    conn = psycopg2.connect(**db_params)
    return conn

In [ ]:
import pandas as pd

df = pd.read_csv('CSV/merged.csv')

# extracts the second string of numbers from the URL
def extract_id(url):
    meet_id = url.split('/')[-3] if url else None
    return meet_id

# Apply the function to the 'event_url' column and replace 'tffrs_meet_id'
df['tffrs_meet_id'] = df['event_url'].apply(extract_id)

# Write the  back to the CSV file
df.to_csv('merged_with_section_id.csv', index=False)

In [ ]:
# This is to merge the two files into one file to then insert sections into the DB

import csv

# Read the first CSV file into a list of dictionaries
with open('CSV/one.csv', 'r') as file:
    reader = csv.DictReader(file, fieldnames=['meet_url', 'event', 'event_url'])
    list_one = list(reader)

# Read the second CSV file into a dictionary
with open('CSV/two.csv', 'r') as file:
    reader = csv.DictReader(file, fieldnames=['meet_name', 'meet_url', 'meet_id', 'date', 'location'])
    dict_two = {row['meet_url']: row for row in reader}

# Merge the data
merged_data = []
for row in list_one:
    if row['meet_url'] in dict_two:
        merged_row = {**row, **dict_two[row['meet_url']]}
        merged_data.append(merged_row)

# write the merged files into a new file
with open('merged.csv', 'w', newline='') as file:
    fieldnames = ['meet_url', 'event', 'event_url', 'meet_name', 'meet_id', 'date', 'location']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(merged_data)

In [ ]:
## this is to add sex to the races based on the url

import csv


with open('merged.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Add the 'sex' field
for row in data:
    if 'Men' in row['event_url']:
        row['sex'] = 'M'
    elif 'Women' in row['event_url']:
        row['sex'] = 'F'
    else:
        row['sex'] = ''

# Write the updated data back to the CSV file
with open('merged.csv', 'w', newline='') as file:
    fieldnames = ['meet_url', 'event', 'event_url', 'meet_name', 'meet_id', 'date', 'location', 'sex']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

In [ ]:
import datefinder
import re

# BUG this does not work properly and gives dates back wrong
def find_date_in_string(input_string):

    input_string = re.sub(r'\s+', ' ', input_string).strip()
    
    # if there is a date range in the string
    match = re.match(r'(\w+ \d+)-(\w+ \d+), (\d{4})', input_string)

    if match:

        start_date_str = f'{match.group(1)}, {match.group(3)}'
        end_date_str = f'{match.group(2)}, {match.group(3)}'

        # Parse the start date
        start_dates = list(datefinder.find_dates(start_date_str))
        if start_dates:
            return start_dates[0].strftime('%Y-%m-%d')
        else:
            print(f"No start date found in string: {input_string}")
            return None
    else:
        # Handle non-range dates like DNS or DNF
        matches = list(datefinder.find_dates(input_string))
        if matches:
            return matches[0].strftime('%Y-%m-%d')
        else:
            print(f"No date found in string: {input_string}")
            return None

In [ ]:
def insert_race(conn, race: Dict):
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO Races (meet_id, meet_name, section, tfrrs_url, date, sex, location, tffrs_meet_id)
            VALUES (nextval('meet_id_seq'), %s, %s, %s, %s, %s, %s, %s)
            ON CONFLICT (tffrs_meet_id) DO UPDATE SET
                meet_name = EXCLUDED.meet_name,
                section = EXCLUDED.section,
                tfrrs_url = EXCLUDED.tfrrs_url,
                date = EXCLUDED.date,
                sex = EXCLUDED.sex,
                location = EXCLUDED.location,
                tffrs_meet_id = EXCLUDED.tffrs_meet_id;
        """, (race['meet_name'], race['event'], race['event_url'], race['date'], race['sex'], race['location'], race['tffrs_meet_id']))
        conn.commit()

In [ ]:
def main():
    conn = connect_db(db_params)

    # Read the merged CSV file into a list of dictionaries
    with open('CSV/merged1.csv', 'r') as file:
        reader = csv.DictReader(file)
        data = list(reader)

    for race in data:
        # Convert the 'date' field to a datetime object
        race['date'] = find_date_in_string(race['date'])
        #print(race)
        insert_race(conn, race)

    conn.close()

In [ ]:
main()